In [ ]:
import torch
torch.cuda.is_available()
print(torch.zeros(1).cuda())

import torch_geometric
import torch_geometric.nn as pyg_nn
from torch_sparse import SparseTensor, matmul
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
print(torch_geometric.__version__)

import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType, OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

import networkx as nx
from deepsnap.hetero_graph import HeteroGraph
import copy
from copy import deepcopy
import pickle

import deepsnap
from deepsnap.graph import Graph
from deepsnap.batch import Batch
from deepsnap.dataset import GraphDataset
from deepsnap.hetero_gnn import forward_op
from deepsnap.hetero_graph import HeteroGraph
from sklearn.metrics import f1_score, roc_auc_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
from pathlib import Path as Data_Path
from os import listdir
from os.path import isfile, join
from itertools import combinations
from tqdm.notebook import tqdm

import graph_tool.all as gt
import json
print("graph-tool version: {}".format(gt.__version__.split(' ')[0]))

In [ ]:
import pickle

with open("../graph_pt_100.pickle", "rb") as f:
    final_graph = pickle.load(f)

In [ ]:
print("Number of vertices:", final_graph.num_vertices())
print("Number of edges:", final_graph.num_edges())

In [ ]:
print('without artists as nodes:')
print("Number of vertices:", final_graph.num_vertices())
print("Number of edges:", final_graph.num_edges())

In [ ]:
# count_pl = sum(1 for v in final_graph.vertices() if final_graph.vp.pl_type[v] == "playlist")
# print(f'num_playlists: {count_pl}')
# count_tr = sum(1 for v in final_graph.vertices() if final_graph.vp.tr_type[v] == "track")
# print(f'num_tracks: {count_tr}')
# count_ar = sum(1 for v in final_graph.vertices() if final_graph.vp.ar_type[v] == "artist")
# print(f'num_artist: {count_ar}')

In [ ]:
print('without artists as nodes:')
count_pl = sum(1 for v in final_graph.vertices() if final_graph.vp.pl_type[v] == "playlist")
print(f'num_playlists: {count_pl}')
count_tr = sum(1 for v in final_graph.vertices() if final_graph.vp.tr_type[v] == "track")
print(f'num_tracks: {count_tr}')
count_ar = sum(1 for v in final_graph.vertices() if final_graph.vp.ar_type[v] == "artist")
print(f'num_artist: {count_ar}')

In [ ]:
### Removal of duplicate nodes from pickle file


# prop_name = "pl_type"

# # Create a dictionary of property values to vertex IDs
# vertex_dict = {}
# for v in final_graph.vertices():
#     value = final_graph.vp[prop_name][v]
#     if value not in vertex_dict:
#         vertex_dict[value] = [int(v)]
#     else:
#         vertex_dict[value].append(int(v))

# # Remove duplicate nodes
# for value, ids in vertex_dict.items():
#     if len(ids) > 1:
#         print(value,ids)
#         # # Merge the duplicate nodes into the first node
#         # first_id = ids[0]
#         # for other_id in ids[1:]:
#         #     final_graph.vertex(first_id).out_edges()[:] = gt.find_edge(final_graph, final_graph.vertex(first_id), final_graph.vertex(other_id))
#         #     final_graph.vertex(first_id).out_edges()[:] = gt.find_edge(final_graph, final_graph.vertex(other_id), final_graph.vertex(first_id))
#         #     final_graph.remove_vertex(final_graph.vertex(other_id))


# print("Number of vertices:", final_graph.num_vertices())
# print("Number of edges:", final_graph.num_edges())

### Largest component

#### Graph-tool

In [ ]:
largest_comp = gt.extract_largest_component(final_graph)
# largest_comp = gt.GraphView(final_graph, vfilt = gt.label_largest_component(final_graph))

In [ ]:
print('no artists:')
print("Number of vertices:", largest_comp.num_vertices()) 
print("Number of edges:", largest_comp.num_edges())

In [ ]:
print('no artists stats:')
print('----------------')
count_pl = sum(1 for v in largest_comp.vertices() if largest_comp.vp.pl_type[v] == "playlist")
print(f'num_playlists: {count_pl}')
count_tr = sum(1 for v in largest_comp.vertices() if largest_comp.vp.tr_type[v] == "track")
print(f'num_tracks: {count_tr}')
count_ar = sum(1 for v in largest_comp.vertices() if largest_comp.vp.ar_type[v] == "artist")
print(f'num_artist: {count_ar}')

In [ ]:
# #### Networkx
# nx_graph = nx.Graph()
# for node in final_graph.vertices():
#     nx_graph.add_node(int(node))
# for edge in final_graph.edges():
#     nx_graph.add_edge(int(edge.source()), int(edge.target()))

# from networkx.algorithms.components import is_connected
# is_connected(nx_graph)

# largest_cc = max(nx.connected_components(nx_graph), key=len)
# nx_largest_comp = nx.Graph(nx_graph.subgraph(largest_cc))
# print('Num nodes:', nx_largest_comp.number_of_nodes(), '. Num edges:', nx_largest_comp.number_of_edges())

### Dataframe generation for EDA

In [ ]:
def generate_playlist_df(playlist_id, playlist_json):
    cols = ['plst_id', 'plst_name', 'track_id', 'track_name']
    data_col = []
    plst_name = playlist_json['name']
    
    for track in playlist_json['tracks']:
        track_id = ''.join(track['track_uri'].split(':')[2:])
        data_col.append([f'plst_{playlist_id}', plst_name, track_id, track['track_name']])
        
    plst_df = pd.DataFrame(data=data_col, columns=cols)
    return plst_df

def generate_file_df(start_index, file_json):
    dfs = []
    for i, plst_json in enumerate(file_json['playlists']):
        dfs.append(generate_playlist_df(start_index + i, plst_json))
        
    df_sum = pd.concat(dfs)
    return df_sum

def generate_spotify_df(data_path, start_index=None, end_index=None):
    assert (start_index is None and end_index is None) or \
        (start_index is not None and end_index is not None), 'Set both or none indices.'
    json_names = [f for f in listdir(data_path) if isfile(join(data_path, f)) and '.json' in f]
    
    num_playlists = start_index if start_index is not None else 0
    section = json_names if start_index is None else json_names[start_index:end_index]
    dfs = []
    
    for file_name in tqdm(section, desc='Files processed: ', unit='files', total=len(section)):
        with open(join(data_path, file_name)) as json_file:
            data = json.load(json_file)
            
        dfs.append(generate_file_df(num_playlists, data))
        num_playlists += len(data['playlists'])
        
    df_total = pd.concat(dfs)
    return df_total

# data_path = '../data/'
# df = generate_spotify_df(data_path, 0, 100)

### N-hop neighbourhood

In [ ]:
pass

### Deepsnap

In [ ]:
g_nx = nx.Graph()

g_nx = nx.Graph()
for v in largest_comp.vertices():
    g_nx.add_node(int(v))

# Add edges to NetworkX graph in batches
batch_size = 100000
edges = [(int(e.source()), int(e.target())) for e in largest_comp.edges()]
num_edges = len(edges)
for i in range(0, num_edges, batch_size):
    batch_edges = edges[i:i+batch_size]
    g_nx.add_edges_from(batch_edges)

        
# Create a DeepSNAP graph from NetworkX graph
ds_graph = Graph(g_nx)

In [ ]:
print("Number of nodes:", nx_graph.number_of_nodes())
print("Number of edges:", nx_graph.number_of_edges())

In [ ]:
print("Number of vertices:", largest_comp.num_vertices()) 
print("Number of edges:", largest_comp.num_edges())